In [1]:
import numpy as np
import os

data_path = os.getenv("HOME")+'/aiffel/speech_recognition/data/speech_wav_8000.npz'
speech_data = np.load(data_path)    

#스피치 데이터를 불러와줍니다. waveform형식의 데이터를 다운받았습니다. yes, no 같은 짧은 단어들에 라벨이 달려있어서 
#간단한 분류기 학습 모델을 만드는데 적합하다고 합니다.

오디오셋 구성 (Train, Test 데이터 구성)

구별해야할 레이블에는 
**['yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go' ]** 이 있습니다.


In [2]:
#라벨을 인덱싱 해줍니다.
#딕셔너리를 만들어 new_label_value라고 이름 붙여주고 인덱싱 해주었다.
#Q. 그냥 enumerate 함수만 사용해도 인덱싱이 될 텐데 왜 굳이 '새로운 딕셔너리'를 생성해주는걸까?
target_list = ['yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go']

label_value = target_list
label_value.append('unknown')
label_value.append('silence')

print('LABEL : ', label_value)

new_label_value = dict()
for i, l in enumerate(label_value):
    new_label_value[l] = i
label_value = new_label_value

print('Indexed LABEL : ', new_label_value)

LABEL :  ['yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go', 'unknown', 'silence']
Indexed LABEL :  {'yes': 0, 'no': 1, 'up': 2, 'down': 3, 'left': 4, 'right': 5, 'on': 6, 'off': 7, 'stop': 8, 'go': 9, 'unknown': 10, 'silence': 11}


In [4]:
#데이터를 분리해줍니다. 

temp = []
for v in speech_data["label_vals"]:
    temp.append(label_value[v[0]])
label_data = np.array(temp)


from sklearn.model_selection import train_test_split

sr = 8000
train_wav, test_wav, train_label, test_label = train_test_split(speech_data["wav_vals"], 
                                                                label_data, 
                                                                test_size=0.1,
                                                                shuffle=True)
print(train_wav)

train_wav = train_wav.reshape([-1, sr, 1]) # add channel for CNN
test_wav = test_wav.reshape([-1, sr, 1])
print("✅")

[[ 3.29382019e-03  4.73433267e-03  8.27394761e-05 ...  2.57896585e-03
   2.08522100e-03 -1.77786360e-03]
 [-7.89344485e-05 -3.73922783e-04 -4.78024187e-04 ...  3.69290181e-04
   2.79870146e-04  2.77746964e-04]
 [ 3.29129561e-03  4.73915599e-03  1.10617155e-04 ...  2.57171458e-03
   2.12259637e-03 -1.78144639e-03]
 ...
 [ 3.68149928e-03  5.37393196e-03  1.92978172e-04 ...  2.02738284e-03
   1.77311827e-03 -2.18975171e-03]
 [ 3.07198381e-03  4.35279449e-03 -3.54647986e-04 ...  2.00132374e-03
   1.53117185e-03 -2.38927803e-03]
 [ 9.92913963e-04 -6.73621136e-04 -2.14048848e-03 ...  3.86722223e-03
   3.08308657e-03  3.95817123e-03]]
✅


In [5]:
print("train data : ", train_wav.shape)
print("train labels : ", train_label.shape)
print("test data : ", test_wav.shape)
print("test labels : ", test_label.shape)
print("✅")

train data :  (45558, 8000, 1)
train labels :  (45558,)
test data :  (5062, 8000, 1)
test labels :  (5062,)
✅


Hyper-parameters setting
학습을 위한 하이퍼파라미터를 설정해줍니다.모델 체크포인트 저장을 위한 체크포인트의 경로를 설정해줍니다.
후에 모델 체크포인트 Callback 함수를 설정하거나, 모델을 불러올때 사용합니다.



In [6]:
batch_size = 32
max_epochs = 1

# the save point
checkpoint_dir = os.getenv('HOME')+'/aiffel/speech_recognition/models/wav'

checkpoint_dir

'/home/aiffel-dj57/aiffel/speech_recognition/models/wav'

## 정신적으로 힘든일이 생겨서 진행을 많이 못했어요 꼼수로라도 일단 제출합니다.